## Encoding and Decoding custom type objects

In [1]:
import json
import numpy as np
from dataclasses import dataclass

In [2]:
@dataclass
class Order:
    name: str
    age: int
    weight: int

data = [
    "data of Class Order",
    Order("Kamran", 20, 65),
    Order("Mahtab", 22, 70),
    np.array([500, 600, 700]),
]
# encoded = json.dumps(data)
# TypeError: Object of type Order is not JSON serializable

In [3]:
# when json wants to raise the above error, call this function
def custom_encoder(obj):
    if isinstance(obj, Order):
        return {
            "__order__":{
                "name": obj.name,
                "age": obj.age,
                "weight": obj.weight,
            }
        }
    if isinstance(obj, np.ndarray):
        return {"__numpy__": obj.tolist()}

def custom_decoder(obj):
    if "__order__" in obj:
        order = obj["__order__"]
        return Order(order["name"], order["age"], order["weight"])
    if "__numpy__" in obj:
        numpy_obj = obj["__numpy__"]
        return np.array(numpy_obj)
    return obj

In [4]:
print(f"type of original: {type(data)}")
print(data)
print("=" * 50)

encoded = json.dumps(data, default=custom_encoder)
print(f"type of encoded: {type(encoded)}")
print(encoded)
print("=" * 50)

decoded = json.loads(encoded, object_hook=custom_decoder)
print(f"type of decoded: {type(decoded)}")
print(decoded)
print("=" * 50)

type of original: <class 'list'>
['data of Class Order', Order(name='Kamran', age=20, weight=65), Order(name='Mahtab', age=22, weight=70), array([500, 600, 700])]
type of encoded: <class 'str'>
["data of Class Order", {"__order__": {"name": "Kamran", "age": 20, "weight": 65}}, {"__order__": {"name": "Mahtab", "age": 22, "weight": 70}}, {"__numpy__": [500, 600, 700]}]
type of decoded: <class 'list'>
['data of Class Order', Order(name='Kamran', age=20, weight=65), Order(name='Mahtab', age=22, weight=70), array([500, 600, 700])]
